# De lo *difícil* a lo *fácil*: conectar y trabajar con bases de datos usando **sqlite3**, **pandas** y **SQLAlchemy**

Este cuaderno sirve como **referencia** para que veas dos formas de trabajar con bases de datos y, después, cómo **generalizar** el patrón a otros motores:

1. **Camino “manual” con `sqlite3`**: hacemos `DROP/CREATE/INSERT/SELECT` y gestionamos `cursor` y `commit`. Así se entiende qué pasa *por debajo*.
2. **Camino “sencillo” con SQLAlchemy + pandas**: repetimos el flujo desde cero pero ahora **sin** manejar cursor ni DDL/INSERT a mano: `create_engine`, `to_sql`, `read_sql`.
3. **Generalización a MySQL y PostgreSQL**: mismo código de lectura/escritura; **solo cambia la URL de conexión**.

Al final, verás **beneficios** y **limitaciones** de este enfoque, más un mini **troubleshooting**.


## Tabla de contenidos
1. [Parte A — SQLite *sin* SQLAlchemy (camino manual)](#parte-a)
2. [Parte B — SQLite *con* SQLAlchemy + pandas (camino sencillo)](#parte-b)
3. [Parte C — Misma idea con MySQL](#parte-c)
4. [Parte D — Misma idea con PostgreSQL](#parte-d)
5. [Beneficios, limitaciones y notas útiles](#beneficios)


---
<a id="parte-a"></a>
# Parte A — SQLite *sin* SQLAlchemy (camino manual)

**Objetivo:** entender *qué hay que hacer a bajo nivel* cuando no usamos SQLAlchemy. Trabajaremos con `sqlite3` para crear una tabla, insertar datos y consultarlos.

**Qué veremos:**
- Conexión a un fichero `.db`
- Uso de `cursor` y `execute(...)`
- DDL/INSERT “a mano” (`DROP/CREATE/INSERT`)
- `commit()` para confirmar cambios
- `SELECT` y lectura con `fetchall()`


In [ ]:
import os
import sqlite3
import pandas as pd  # Solo para mostrar luego cómo se vería en DataFrame


In [ ]:
# Conexión a un fichero SQLite (se crea si no existe)
con = sqlite3.connect(r'./bases.datos/sqlite_db_story.db')
con


In [ ]:
# (1) Limpiar si existe: DROP TABLE
drop_query = "DROP TABLE IF EXISTS projects;"
cursor = con.cursor()
cursor.execute(drop_query)
con.commit()  # Confirmamos cambios DDL


In [ ]:
# (2) Crear tabla: CREATE TABLE
create_query = '''
CREATE TABLE IF NOT EXISTS projects (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    begin_date TEXT,
    end_date TEXT
);
'''
cursor = con.cursor()
cursor.execute(create_query)
con.commit()


In [ ]:
# (3) Insertar datos: INSERT
insert_query = '''
INSERT INTO projects (id, name, begin_date, end_date) VALUES
(1, 'Juan',  '2024-01-01', '2024-06-30'),
(2, 'Ana',   '2024-03-15', '2024-09-15'),
(3, 'Lucía', '2024-05-01', NULL);
'''
cursor = con.cursor()
cursor.execute(insert_query)
con.commit()  # ¡IMPORTANTE!


In [ ]:
# (4) Consultar: SELECT + fetchall()
select_query = "SELECT id, name, begin_date, end_date FROM projects;"
cursor = con.cursor()
cursor.execute(select_query)
rows = cursor.fetchall()
rows  # Lista de tuplas


In [ ]:
# (5) (Opcional) Verlo como DataFrame (misma conexión, sin SQLAlchemy)
cols = ['id', 'name', 'begin_date', 'end_date']
pd.DataFrame(rows, columns=cols)


**Conclusión de la Parte A**  
Trabajar sin SQLAlchemy **funciona**, pero hay que ocuparse de:
- crear/eliminar tablas,
- preparar `INSERT`s,
- acordarse del `commit()`,
- y gestionar el `cursor`.

Ahora repetimos **desde cero** con **SQLAlchemy + pandas**, donde parte de todo esto se simplifica mucho.


---
<a id="parte-b"></a>
# Parte B — SQLite *con* SQLAlchemy + pandas (camino sencillo)

**Objetivo:** repetir el caso anterior pero:
- Conexión con `create_engine('sqlite:///ruta.db')`
- Crear/poblar la tabla **directamente** desde un `DataFrame` con `.to_sql(...)`
- Leer con `pd.read_sql(...)`
- Sin `cursor`, sin `commit` explícito, y sin escribir `CREATE/INSERT` a mano


In [ ]:
from sqlalchemy import create_engine


In [ ]:
# Nueva base/tabla para partir de cero
engine_sqlite = create_engine('sqlite:///./bases.datos/sqlite_db_story_sqlalchemy.db')
engine_sqlite


In [ ]:
# Creamos un DataFrame de ejemplo y lo escribimos a la BD
df_projects = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Juan', 'Ana', 'Lucía'],
    'begin_date': ['2024-01-01', '2024-03-15', '2024-05-01'],
    'end_date': ['2024-06-30', '2024-09-15', None]
})

# .to_sql crea la tabla si no existe y hace los INSERTs por nosotros
df_projects.to_sql('projects', con=engine_sqlite, if_exists='replace', index=False)


In [ ]:
# Leemos con pandas
pd.read_sql("SELECT * FROM projects", con=engine_sqlite)


In [ ]:
# Podemos añadir filas fácilmente
df_more = pd.DataFrame({
    'id': [4],
    'name': ['Marcos'],
    'begin_date': ['2024-06-01'],
    'end_date': [None]
})
df_more.to_sql('projects', con=engine_sqlite, if_exists='append', index=False)

pd.read_sql("SELECT * FROM projects ORDER BY id", con=engine_sqlite)


**Comentarios rápidos (Parte B):**
- `.to_sql(..., if_exists='replace')` **borra y recrea** la tabla (¡cuidado!).   Usa `'append'` para añadir filas.
- **No** hemos escrito `CREATE/INSERT` ni gestionado `cursor` o `commit` a mano.
- En tablas reales quizá quieras **tipos de datos más precisos** o **claves**;
  `.to_sql` hace una inferencia básica. Para control avanzado puedes crear la tabla antes o usar tipos de SQLAlchemy, pero aquí **priorizamos sencillez**.


---
<a id="parte-c"></a>
# Parte C — Misma idea con **MySQL**

**Supuesto:** MySQL corre en Docker y escucha en `localhost:3306` (ajusta a tu entorno). Con **pandas + SQLAlchemy** el código de lectura/escritura es **igual**; solo cambia la **URL de conexión**.

> Si tu driver no está instalado, podrías necesitar: `pip install sqlalchemy mysqlclient`


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

mysql_user = "root"
mysql_password = "rootpass"
mysql_host = "localhost"
mysql_port = 3306
mysql_db = "test_db"

mysql_url = f"mysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_db}"
engine_mysql = create_engine(mysql_url)
engine_mysql


In [ ]:
# Escribimos un DataFrame en MySQL
df_mysql = pd.DataFrame({'a': [10, 20], 'b': ['hola', 'caracola']})
df_mysql.to_sql('demo_mysql', con=engine_mysql, if_exists='append', index=False)

# Y leemos igual que en SQLite
pd.read_sql("SELECT * FROM demo_mysql", con=engine_mysql)


---
<a id="parte-d"></a>
# Parte D — Misma idea con **PostgreSQL**

**Supuesto:** PostgreSQL corre en Docker y escucha en `localhost:5432`. Para el driver de PostgreSQL solemos usar `psycopg2`.

> Si hace falta: `pip install sqlalchemy psycopg2-binary`


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

pg_user = "postgres"
pg_password = "postgres"
pg_host = "localhost"
pg_port = 5432
pg_db = "test_db"

pg_url = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
engine_pg = create_engine(pg_url)
engine_pg


In [ ]:
# Misma operación: escribir y leer
df_pg = pd.DataFrame({'x': [1, 2, 3], 'y': ['A', 'B', 'C']})
df_pg.to_sql('demo_pg', con=engine_pg, if_exists='append', index=False)

pd.read_sql("SELECT * FROM demo_pg", con=engine_pg)


---
<a id="beneficios"></a>
# Beneficios, limitaciones y notas útiles

## ¿Por qué este enfoque funciona tan bien?
- **Portabilidad:** con `pandas + SQLAlchemy` el **patrón es idéntico** entre SQLite, MySQL y PostgreSQL;   cambia la **URL de conexión** y poco más.
- **Simplicidad:** `to_sql`/`read_sql` cubren el 80% de casos típicos de clase/prototipado.
- **Rapidez de desarrollo:** menos SQL manual, menos `cursor`, menos `commit` explícito.

## ¿Qué limitaciones tiene?
- **Tipos y claves:** `.to_sql` hace *inferencia básica*. Si necesitas claves primarias compuestas, índices,   tipos muy específicos, etc., tendrás que **crear la tabla** tú mismo/a (DDL) o usar tipos de SQLAlchemy.
- **Control fino:** para transacciones complejas o SQL avanzado, escribir queries manuales sigue siendo útil.
- **Rendimiento en volumen:** para cargas muy grandes conviene considerar `chunksize=...`, COPY (en Postgres),   o utilidades específicas del motor.

## Consejos y resolución de problemas
- **Drivers:** si MySQL o Postgres no conectan, revisa que tengas el driver instalado (`mysqlclient` / `psycopg2-binary`).  
- **Docker:** comprueba que los contenedores están levantados y los puertos mapeados (`3306`, `5432`).  
- **Credenciales/host:** asegúrate de usuario, contraseña, host/IP y DB correctos.  
- **`if_exists`:** `'append'` añade filas; `'replace'` borra y recrea la tabla (¡ojo!).  
- **Seguridad:** no dejes credenciales en claro en proyectos reales; usa `.env`/variables de entorno.
